In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("Number of Rating for every movie")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Movies_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/movie.csv")
      )

newdf=(Ratings_Dataframe.groupBy("movieId").count()
                        .join(Movies_Dataframe,Movies_Dataframe.movieId==Ratings_Dataframe.movieId)
                        .drop(Movies_Dataframe.movieId)
                        .drop(Movies_Dataframe.genres)
                        .drop(Ratings_Dataframe.movieId)
                        .withColumnRenamed('count', 'NumberOfRatings')
      )
newdf.sort("NumberOfRatings",ascending=False).show(n=5,truncate=0)
stagemetrics.end()

+---------------+--------------------------------+
|NumberOfRatings|title                           |
+---------------+--------------------------------+
|67310          |Pulp Fiction (1994)             |
|66172          |Forrest Gump (1994)             |
|63366          |Shawshank Redemption, The (1994)|
|63299          |Silence of the Lambs, The (1991)|
|59715          |Jurassic Park (1993)            |
+---------------+--------------------------------+
only showing top 5 rows



In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 7
numTasks => 216
elapsedTime => 52747 (53 s)
stageDuration => 50433 (50 s)
executorRunTime => 147411 (2.5 min)
executorCpuTime => 60383 (1.0 min)
executorDeserializeTime => 14283 (14 s)
executorDeserializeCpuTime => 3087 (3 s)
resultSerializationTime => 485 (0.5 s)
jvmGCTime => 2178 (2 s)
shuffleFetchWaitTime => 41 (41 ms)
shuffleWriteTime => 530 (0.5 s)
resultSize => 1388194 (1355.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 2285992000
recordsRead => 40055086
bytesRead => 1384480482 (1320.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 113768
shuffleTotalBlocksFetched => 1200
shuffleLocalBlocksFetched => 1200
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 1466752 (1432.0 KB)
shuffleLocalBytesRead => 1466752 (1432.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDi